In [86]:
import pandas as pd
import numpy as np

import re
import torch
import faiss

import io
import glob

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM

from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize

In [2]:
df_articles = pd.read_csv('./df_data.csv')
df_articles

,text
0,\nКапитан сборной Аргентины Лионель Месси сооб...
1,\nЛидер сборной Аргентины Лионель Месси вряд л...
2,\nINTERFAX.RU - Аргентинский футболист Лионель...
3,\nЛионель Месси не планирует участвовать в чем...
4,\nКапитан сборной Аргентины по футболу Лионель...
...,...
95,\nТинькофф Банк объявил о запуске Центрального...
96,\n«Тинькофф» запускает частный университет со ...
97,\n14 июня 2023 года «Тинькофф» объявил о запус...
98,\nТинькофф банк запустит частный университет -...


In [38]:
articles_list = df_articles['text'].tolist()

In [4]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum", use_fast=False)

model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

In [6]:
def generate_text(articles):
    WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

    input_ids = tokenizer(
        [WHITESPACE_HANDLER(el) for el in articles],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=1024
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        min_length=700,
        no_repeat_ngram_size=3,
        num_beams=4,
        max_new_tokens=800
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return summary

In [93]:
def token_articles(base_articles):
    tokenized_articles = []
    for i in range(len(base_articles)):
        tokenized_articles.append(word_tokenize(base_articles[i]))
    return tokenized_articles

In [96]:
new_article = generate_text(articles_list[:10])
new_article

'Лионель Месси объявил, что не будет участвовать в следующем чемпионате мира по футболу, который пройдет в Мексике, США и Канаде в 2026 году. Об этом говорится в сообщении, опубликованном на официальном сайте аргентинского футбольного клуба ПСЖ в пятницу, 27 апреля. Это означает, что Месси будет играть в Бразилии в последний чемпионат мира в этом году, когда он будет работать. Для Месси.  Таким была его последняя история. После того, как Месси провел последний турнир. Журналист и футболист уже давно знал, что его фанаты уже поговорили о том, как ему это было именно так. Вот и сам Месси в беседе с Би-би-си. Что делать с Месси, а также о своем последнем матче в футболе, он не поедет на следующий чемпионат, где он не участвует в турнире, в котором он был победителем чемпионата мира в декабре 2022 года, и что будет с ним еще в прошлом году - его последний сезон в карьере и почему его не будут принимать матчи с бразильским клубом ФИФА, Месси отказался от участия в чемпионате, которые пройду

In [97]:
tokenized_new_article_football = word_tokenize(new_article, language="russian")

In [98]:
tokenized_articles_football = token_articles(articles_list[:10]) 

In [99]:
round(meteor_score.meteor_score(tokenized_articles_football, tokenized_new_article_football), 4)

0.2965

In [100]:
new_article_saintp = generate_text(articles_list[10:20])
new_article_saintp

'В Санкт-Петербурге пройдет ежегодный праздник выпускников школ «Алые Паруса» и концерт с участием звезд первой величины, а также фейерверков и пиротехнического шоу. Что ждет петербуржцев и гостей города, рассказывает корреспондент Русской службы Би-би-си Валерий Герасимов. .   . Эти и другие новости - в обзоре bbcrussian.com , которые можно увидеть в этом блоге .. bbc .a  ,.  ( ) . " Петербург - " bbcuzbek.com. , " - bbc.com, - на bbcvietnam.org , в Петербурге - BBC News . ( ). ) В этом году в сентябре и в июне в  Санкт-Франциско, и еще одна тема - концерт, концерт и праздничные мероприятия. Вот что будет с ним в июле прошлого года. ). В прошлом году фестиваль должен состояться в Санкт-Петерском университете , который проходит уже в пятницу, 24 июня, впервые в истории Петербурга, где можно будет наблюдать за праздничным концертом и как будет работать фестиваль, о котором известно уже давно, когда именно в этой стране и почему не было ничего такого, чтобы не пропустить в прошлую неделю

In [101]:
tokenized_new_article_film = word_tokenize(new_article_saintp, language="russian")

In [102]:
tokenized_articles_film = token_articles(articles_list[10:20]) 

In [103]:
round(meteor_score.meteor_score(tokenized_articles_film, tokenized_new_article_film), 4)

0.3038

In [104]:
new_article_music = generate_text(articles_list[20:30])
new_article_music

'Американская поп-певица Лана Дель Рей выпустила новый альбом, который стал самым успешным альбомом в карьере. Об этом говорится в сообщении, опубликованном на официальном сайте музыкального журнала Pitchfork, которое опубликовал корреспондент Би-би-си Джеффри Пайетт . ..   Что известно о новом альбоме певицы и авторе-исполнителе и ее новой пластинке. bbc .co.uk  ., . " . ( ) " - bbc.com , а также , " , в этом году , на этом месте , не было ни одного альбома , с ним можно и не писать, и в том числе и на сайте журнала, где именно его можно увидеть , по ее словам, является одним из самых успешных альбомов в истории музыки и музыке , которая была выпущена в течение десятилетий, когда она была создана и вышла в прошлом году и сейчас, впервые с момента ее выхода в свет и после того, как его выпустили в ноябре прошлого года, если бы вы могли сказать о том, что ее песни были выпущены в ближайшее десятилетие, - в следующем году - это лучший альбом и самый успешный альбом в ее истории , чтобы с

In [105]:
tokenized_new_article_music = word_tokenize(new_article_music, language="russian")

In [106]:
tokenized_articles_music = token_articles(articles_list[20:30]) 

In [107]:
round(meteor_score.meteor_score(tokenized_articles_music, tokenized_new_article_music), 4)

0.1831

In [83]:
new_article = generate_text(articles_list[30:40])
new_article

'Американский режиссер Мартин Скорсезе объявил о выходе нового фильма о расследовании убийств индейцев в штате Оклахома в США. Что же будет с ним с точки зрения расследования ФБР, и почему его роль стала одной из самых амбициозных и необычных ролей?   . . В очередной подборке Би-би-си рассказывает обозреватель BBC Culture о том, что будет происходить в фильме ..   BBC . ( .) . " Это история , ., - именно , " - рассказывает его автор и режиссер, а уже давно, когда ему стало известно, что в этом году его роман и фильм, которые его вдохновили, чтобы его создать и рассказать, как будет работать и как можно скорее, о чем рассказывает корреспондент BBC Culture. ), который рассказывает историю о своем новом фильме, который будет выходить на экраны в следующем году, где он будет рассказывать о роли Джеймса Бонда в истории американского бюро расследований ФБР. Вот уже несколько десятилетий режиссер рассказывает, кто будет представлять собой главную роль в новом триллере Мартина Скорсезe , в кот

In [92]:
tokenized_new_article_film = word_tokenize(new_article, language="russian")

In [94]:
tokenized_articles_film = token_articles(articles_list[30:40]) 

In [95]:
round(meteor_score.meteor_score(tokenized_articles_film, tokenized_new_article_film), 4)

0.2004

In [109]:
new_article_spacex = generate_text(articles_list[40:50])
new_article_spacex

'Американская компания SpaceX осуществила тестовый запуск многоразовой ракеты Starship, которая должна была выполнить один виток вокруг Земли, и взорвалась в Тихом океане. Об этом говорится в видеосъемке, опубликованной на официальном сайте американского космического агентства SpaceX в понедельник, 17 сентябр . .. Это первый запуск ракеты, когда-либо созданной в мире, запуск которого прошел в непилотируемом режиме. Для нее будет работать. Эксперты будут работать над следующими испытаниями. После того, как взорвался лайнер, а потом - часть ракеты. Однако уже можно было сказать, что ракета будет выполнять два витка вокруг планеты, чтобы совершить пуск в Тихий океан, где должен был совершить вращение корабль Ship 24 и ее носитель Super Heavy будет использоваться для пилотируемых полетов на Луну и Марс, которые должны состояться в следующем году, как ожидается, полет на Марс будет осуществляться уже в ближайшие дни, если ракета не будет участвовать в международном космическом корабле, кото

In [110]:
tokenized_new_article_spacex = word_tokenize(new_article_spacex, language="russian")

In [111]:
tokenized_articles_spacex = token_articles(articles_list[40:50]) 

In [112]:
round(meteor_score.meteor_score(tokenized_articles_spacex, tokenized_new_article_spacex), 4)

0.3903

In [115]:
new_article_fire = generate_text(articles_list[50:60])
new_article_fire

'Около 30 тысяч человек должны эвакуироваться из восточного побережья Канады из-за лесных пожаров, которые стали причиной глобального потепления, сообщает Национальное управление по чрезвычайным ситуациям в Канаде (NHS) и Национальная служба по охране окружающей среды (National Bureau of Statistics). Это означает, что в этом году в канадском восточном побережье бушуют лесные пожары, в результате которых погибли уже более 100 человек. Для них - очень тяжело. Это, вероятно, очень. Почему? Почему и почему? Ответ - не для них. Эксперты считают, что это может стать одним из самых страшных. Вот о том, почему в некоторых регионах канад а т е е в стране не хватает лесного пожара в прошлом году, а также для многих людей, живущих на востоке страны, однако канадские власти предупреждают о масштабных пожарах в восточной части канадской территории, где уже несколько недель бушует лесный пожар, и что они могут быть связаны с изменением климата и изменениями климата, по мнению экспертов, причиной это

In [116]:
tokenized_new_article_fire = word_tokenize(new_article_fire, language="russian")

In [117]:
tokenized_articles_fire = token_articles(articles_list[50:60]) 

In [118]:
round(meteor_score.meteor_score(tokenized_articles_fire, tokenized_new_article_fire), 4)

0.2524

In [119]:
new_article_weather = generate_text(articles_list[60:70])
new_article_weather

'В Москве началось метеорологическое лето на три дня раньше срока, сообщили в центре погоды «Фобос» и метеорологическая служба Московского гидрометцентра. Это означает, что метеорологические лето в Москве пришло на неделю раньше, чем в прошлом году, а в следующем году в Москве -.  Эти и другие новости - в обзоре Русской службы Би-би-си, опубликованные на сайте Кремля. . Что известно о следующ  е . В этом году .. Почему? И как именно и как можно скорее? В Москве уже давно не ясно, когда и в какой-то момент лето будет работать и что в ближайшие годы метеорология в Москве не будет, и почему в этом случае не хватает времени, чтобы повысить температуру воздуха в Москве, по мнению метеорологов, может прийти с опозданием. Вот уже несколько дней метеорологический лето началось с опережением и с того же периода, как было ранее, но уже в 2023 году метеорологи пришли с предупреждением о том, что климатическое лето пришло с осложнением климата, предупреждают синоптики и ученые. Чем же может быть с

In [120]:
tokenized_new_article_weather = word_tokenize(new_article_weather, language="russian")

In [121]:
tokenized_articles_weather = token_articles(articles_list[60:70]) 

In [122]:
round(meteor_score.meteor_score(tokenized_articles_weather, tokenized_new_article_weather), 4)

0.2861

In [124]:
new_article_car = generate_text(articles_list[70:80])
new_article_car

'Японский автомобильный концерн Toyota объявил о планах по созданию электромобилей с дальности хода на одной зарядке, который позволит проехать более 900 миль (1 448 км) без выбросов в атмосферу и меньше выбросов дизельного электромобиля (EV) в ближайшие два-три года, а также. Это может стать одним из самых амбициозных планов японской компании по разработке новых энергетических автомобилей, которые будут. .  .. Для нее уже можно было работать в одном месте. Эксперты и эксперты - одна за одной. Что делать? - в этом. Вопрос - не очень далеко, но вполне очевидно. Японская компания намерена дать отпор в планы о создании електромобилей, однако ее планы могут быть опубликованы на официальном сайте корпорации Япония, которая считает, что электромобили будут оснащены батареями, рассчитанными на почти 1500 км пробега, и в следующем году они начнут выпускать электромобильы, используя литий-ионную батарею, чтобы увеличить дальности пробега в пределах одного заряда, если они будут выпущены из-за т

In [125]:
tokenized_new_article_car = word_tokenize(new_article_car, language="russian")

In [128]:
tokenized_articles_car = token_articles(articles_list[70:80]) 

In [129]:
round(meteor_score.meteor_score(tokenized_articles_car, tokenized_new_article_car), 4)

0.3261

In [131]:
new_article_bitcoin = generate_text(articles_list[80:90])
new_article_bitcoin

'Курс ведущей криптовалюты Bitcoin снизился на 7% после того, как Федеральная резервная система США объявила о повышении процентных ставок в связи с угрозой для экономики страны, которые угрожают американским инвесторам и их рынкам, а также угрозами для компаний. Это означает, что инвесторы ожидают потери в стоимости ведущих криптовалют, Bitcoin и Dogecoin и Cardano. Для инвесторов. .  ..  Таким была новая история. Экономические активы. Эксперты полагают, что "в этом мире уже можно было купить и покупать " . " - это решение . В понедельник, 17 июня, курс доллара опустился более чем на 5% и впервые за последние два года. Курс доллара превысил отметку на 1%. После того как ФРС объявила об ужесточении процентной ставок регулятора США повысили процентные ставки, однако регулятор готовится к ужесточению ставок, чтобы удержать процентные ставки в случае необходимости - повышение курса будет работать уже в ближайшие дни, когда будет объявлено о том, что курс будет ухудшаться уже на фоне решен

In [132]:
tokenized_new_article_bitcoin = word_tokenize(new_article_bitcoin, language="russian")

In [133]:
tokenized_articles_bitcoin = token_articles(articles_list[80:90]) 

In [134]:
round(meteor_score.meteor_score(tokenized_articles_bitcoin, tokenized_new_article_bitcoin), 4)

0.2683

In [136]:
new_article_tinkoff = generate_text(articles_list[90:100])
new_article_tinkoff

'Российская компания «Тинькофф» объявила об открытии частного университета, который будет специализироваться на математике и компьютерных науках. Это первое в России частное высшее учебное заведение на основе STEM-модели (Science, Technology, Engineering, Engineering), mathematics, technology, engineering, engineering. Что стоит за концепцией.  BBC News Украина рассказывает о том, как будет развиваться этот вуз и будет ли он работать для сотрудников, .   .. . BBC  Kunis. - bbc.com  ,  bbc .com, bbcrussian.co.uk , " bbcuzbek.com. ) . " Школы, которые уже давно оказываются на рынке, а вот уже несколько лет, когда в России уже существует множество вузов, где можно работать, и почему не хватает вузов для открытого рынка и что будет с ними работать?", - рассказывает корреспондент BBC Capital , которая изучает проблемы образования в российской экономике , что может быть связано с проблемами образования и какие будут шансы на повышение эффективности образовательного потенциала в этом ведомств

In [137]:
tokenized_new_article_tinkoff = word_tokenize(new_article_tinkoff, language="russian")

In [138]:
tokenized_articles_tinkoff = token_articles(articles_list[90:100]) 

In [139]:
round(meteor_score.meteor_score(tokenized_articles_tinkoff, tokenized_new_article_tinkoff), 4)

0.2717

### Проверка на тестовых данных

In [171]:
def get_articles(path):
    articles = []
    for file in glob.glob(path +"*.txt"):
        
        with io.open(file, mode='r', encoding='utf-8') as f:
            text = f.read()
            articles.append(text)
        
    return articles

In [159]:
articles_nornikel = get_articles(path='./tests/1/')

In [161]:
new_article_nornikel = generate_text(articles_nornikel[:10])
new_article_nornikel

'Компания «Норникель» оштрафована на 150 млрд рублей за утечку дизельного топлива на ТЭЦ-3 в Норильском крае. Это крупнейший подобный штраф в российской истории. Акции компании теряют около 3% на Московской бирже, а главный акционер проекта «Сахалин- 2» .  Эти и другие новости - в обзоре Русской службы Би-би-си, опубликованные в понедельник на сайте компании.  . "Норник Kuni .. ( ) . (.) ,  ,. , ( bbc.com  ), а в этом году - bbcrussian.co.uk ). Это означает, что компания будет штрафовать в ближайшее время. Российские банки и банки не смогут купить деньги, которые можно купить в РФ, и не только в России, но и в России. ) Росприроднадзор объявил о штрафе, выписанном ведомством, когда будет объявлено о выплате ущерба. Эксперты считают, что убытки от аварии могут быть выплачены только иностранным акционерам, которым придется выплатить ущерб. Нефтяные компании уже заявили о готовности выплатить все начисленные штрафы, но не намерены выплачивать их. Что будет с ними и как выяснилось, кто буд

In [162]:
tokenized_new_article_nornikel = word_tokenize(new_article_nornikel, language="russian")

In [163]:
tokenized_articles_nornikel = token_articles(articles_nornikel[:10]) 

In [164]:
round(meteor_score.meteor_score(tokenized_articles_nornikel, tokenized_new_article_nornikel), 4)

0.244

In [216]:
articles_pdk = get_articles(path='./tests/2/')

In [217]:
new_article_pdk = generate_text(articles_pdk)
new_article_pdk

'ПДК по сероводороду в городе Бузулук в Оренбургской области в 27 раз превышен на 27 раз. Об этом сообщили в министерстве экологии и чрезвычайных ситуациях Оренбургского областного государственного агентства (Роспотребнадзор) со ссылкой на пресс-службу МЧС России и регионального управления МВД.. . Это произошло после того, как в Бузулуке было зафиксировано 27 превышения ПДК, сообщает пресс-служба МВД .. По факту.  . " bbc.co.uk. - bbcrussian.com. ) . В одной из районов города была зафиксована превышение ПДК . в одной и другой стране была превышена ПДК. Российские власти считают, что загрязнение может быть связано с " химическими и экологическими ресурсами. Информацию о причинах загрязнения в Бузулуке не подтвердили. Проверка идет о нарушении законодательства о загрязнении воды в районе Бузулука, где уже 27 раз было превышено ПДК в 27 раза, а еще 27 раз были зафиксированы выбросы. Количество выбросов в прошлом году превысило 27 раз, и в этом году было зарегистрировано 27 новых случаев н

In [218]:
tokenized_new_article_pdk = word_tokenize(new_article_pdk, language="russian")

In [219]:
tokenized_articles_pdk = token_articles(articles_pdk) 

In [220]:
round(meteor_score.meteor_score(tokenized_articles_pdk, tokenized_new_article_pdk), 4)

0.2457

In [221]:
articles_irkutsk = get_articles(path='./tests/3/')

In [222]:
new_article_irkutsk = generate_text(articles_irkutsk)
new_article_irkutsk

'Следственный комитет России возбудил уголовное дело по факту получения взятки в 150 тысяч рублей от гендиректора ООО „ Восход". Об этом сообщили в пресс-службе Следственного комитета России в приангарском районе Приангарья, сообщает пресс-служба ведомства. Глава госинспектора труда задержали за получение .. . После несчастного случая погибли два человека, еще двое получили ранения, сообщают в Следственном комитете России. Это   . " . ( .) .( .). ) " , (  ) , " )" , - говорили по делу о " коррупции  , в этом году прокуратура РФ не стала работать по этому делу, но его обвиняют в получении взяток и о незаконном хранении денег в ходе несчастных случаев в Иркутской области, где были выявлены нарушения по технике безопасности и охране труда, а также о том, что он получил взятку от руководителя ООО “ Восход ”, - говорится в сообщении СКР , что ему предъявлено обвинение по подозрению в причастности к несчастному случаю , который произошел в прошлом году, когда в результате аварии погибли двое

In [223]:
tokenized_new_article_irkutsk = word_tokenize(new_article_irkutsk, language="russian")

In [224]:
tokenized_articles_irkutsk = token_articles(articles_irkutsk) 

In [225]:
round(meteor_score.meteor_score(tokenized_articles_irkutsk, tokenized_new_article_irkutsk), 4)

0.3803

In [179]:
articles_groupIB = get_articles(path='./tests/4/')

In [180]:
new_article_groupIB = generate_text(articles_groupIB)
new_article_groupIB

'Хакер Fxmsp, причастный к продаже данных компаний в разных странах мира, является одним из наиболее опасных преступников в своей среде, сообщили в компании Group-IB Threat Intelligence. Эксперты считают, что хакер продолжает свою деятельность по взлому сетей компаний и все еще представляют опасность.  . BBC News Украина рассказывает о том, как можно обеспечить защиту от киберпреступности компаний, а также о .. . , . ( ) . " ) " - " , "  ( bbc.com   ). " " . Ранее о " bbc .co. a. \' С т е " е т " т в " л е" . Сейчас его история . Однако в одной из самых опасных " террористических группах  , в том числе и в России, где в прошлом году был зафиксирован рост предложений, связанных с продажей доступа к корпоративным сетям , которая была организована хакерской группой FxMSP  ", - говорится в новом отчете о хакерском сообществе Fxmb е, которое было создано в 2017 году, когда хакер скомпрометировал корпорации в разные уголки мира . По данным экспертов, его деятельность стала самым опасным прес

In [181]:
tokenized_new_article_groupIB = word_tokenize(new_article_groupIB, language="russian")

In [182]:
tokenized_articles_groupIB = token_articles(articles_groupIB) 

In [183]:
round(meteor_score.meteor_score(tokenized_articles_groupIB, tokenized_new_article_groupIB), 4)

0.2432

In [187]:
articles_mid = get_articles(path='./tests/5/')

In [188]:
new_article_mid = generate_text(articles_mid)
new_article_mid

'Министерство иностранных дел России выставило на продажу базу данных российских туристов за биткоины, сообщили в пресс-службе ведомства. В ведомстве отрицают, что они получили сведения о россиянах, однако не знают, когда удастся восстановить доступ к аккаунту МИД Марии Захаровой. Однако  российская служба Би-би-си подтвердила факт взлома аккаунта, который в настоящее время доступен для просмотра. Журналисты BBC .. . bbc.co.uk . "Сеть "Подъем" - bbcrussian.com , которые опубликовали в одном из аккаунтов. Российские хакеры пытались купить базу, а в этом году они выставили ее на продажи. Эксперты уверяют, что базу можно купить за криптовалюту. Это может стать одним из самых распространенных в стране событий в ближайшие месяцы. Что известно о том, что это может произойти, и что ведомство не знает, кто именно может купить данные для российского МИДа, оказались в критическом состоянии и как можно быстрее и сколько можно заплатить за эту информацию и почему это происходит и в чем причина это

In [189]:
tokenized_new_article_mid = word_tokenize(new_article_mid, language="russian")

In [190]:
tokenized_articles_mid = token_articles(articles_mid) 

In [191]:
round(meteor_score.meteor_score(tokenized_articles_mid, tokenized_new_article_mid), 4s

0.3246

In [192]:
articles_australia = get_articles(path='./tests/6/')

In [193]:
new_article_australia = generate_text(articles_australia)
new_article_australia

'Австралия увеличит траты на киберб безопасность на $1 млрд из-за атак Китая, сообщили в правительстве страны. Это крупнейший в истории выделение бюджета на обеспечение кибербезопасности страны, который будет вынужден наращивать расходы на защиту экономики, национальной безопасности и суверенитета страны в ближа.  .  Эти и другие новости - в обзоре Би-би-си, опубликованном на сайте правительства Австралии. .. bbc.co.uk . "В этом году будет известно, что в стране уже более одной и одной из самых масштабных и самых крупных кибератаках в мире. Эксперты считают, что Китай - одна из крупнейших кибератак на планете, и в этом смысле не может быть связано с этим положением. Австралия планирует выделить дополнительные деньги на нейтрализацию кибератак из зарубежных стран, а также на создание 500 рабочих мест в управлении радиотехнической обороны страны", - говорится в сообщении, где правительство считает, что это может привести к тому, что киберактивность может оказаться в числе главных источни

In [194]:
tokenized_new_article_australia = word_tokenize(new_article_australia, language="russian")

In [195]:
tokenized_articles_australia = token_articles(articles_australia) 

In [196]:
round(meteor_score.meteor_score(tokenized_articles_australia, tokenized_new_article_australia), 4)

0.3505

In [226]:
articles_social = get_articles(path='./tests/7/')

In [227]:
new_article_social = generate_text(articles_social)
new_article_social

'В текущем году в Казахстане планируется реализовать свыше 2 тысяч социальных проектов на общую консолидированную сумму почти 18 млрд тенге, сообщили в министерстве информации и общественного развития страны. В этом году грантовое финансирование будет увеличиваться в более чем 9 раз, с 208 млн до 1,8 млрд. . Это может стать одним из крупнейших в стране социальных projectов в рамках государственного социального заказа и грантового финансирования.   .. ,  ,.  Эти и.о. bbc.co.uk , " bbcrussian.com  \' . " - bbcuzbek.com. ) . Однако в ближайшее время будет известно, какие меры будут использовать для того, чтобы обеспечить системное развитие гражданского общества. Для этого будет выделено 18 миллионов казахстанцев, которые потратит на информационный госзаказ, а также грантов на социальные проекты , который будет реализовать в следующем году, когда будет объявлено о создании новых инициатив для обеспечения системы гражданского развития , в том числе и в сфере образования и науки, которая буд

In [228]:
tokenized_new_article_social = word_tokenize(new_article_social, language="russian")

In [229]:
tokenized_articles_social = token_articles(articles_social) 

In [230]:
round(meteor_score.meteor_score(tokenized_articles_social, tokenized_new_article_social), 4)

0.3572

In [199]:
articles_kz = get_articles(path='./tests/8/')

In [200]:
new_article_kz = generate_text(articles_kz)
new_article_kz

'Из-за пандемии коронавируса доходы казахстанских СМИ могут сократиться до 60%, сообщили в министерстве информации и общественного развития республики. По данным министерства, доходы СМИ сократятся до 62%, а коммерческие теле- и радиоканалы до 31 декабря будут освободить от налогов. Это уже означает, что их доходы могут уменьшиться до 40%, сообщают агентства со ссылкой на источники в СМИ. .  .. bbc.co.uk ., . " - " . СМИ - bbc .co . ( bbcuzbek.com  ) , - это только одна из самых маленьких в стране, где можно работать и меньше, чем обычно, и несколько десятков тысяч людей, которые работают в сфере рекламы. - В Казахстане оценили потерю доходов СМИ в связи с пандемией коронавируса, однако, по их мнению, они могут потерять 60% и сократить доходы от рекламной деятельности. ), - говорится в отчете о доходах СМИ, который опубликовала пресс-служба казахского правительства, которое было опубликовано на сайте ведомства. Для СМИ это может стать одним из крупнейших экономических трудностей в реги

In [201]:
tokenized_new_article_kz = word_tokenize(new_article_kz, language="russian")

In [202]:
tokenized_articles_kz = token_articles(articles_kz) 

In [203]:
round(meteor_score.meteor_score(tokenized_articles_kz, tokenized_new_article_kz), 4)

0.3227

In [206]:
articles_karelia = get_articles(path='./tests/9/')

In [207]:
new_article_karelia = generate_text(articles_karelia)
new_article_karelia

'На прошлой неделе в Карелии выпал желтый снег, после чего стали массово гибнуть птицы. Местные власти заявили, что экологическая катастрофа в городе в норме. Власти считают, что все экологические показатели в Норме, однако не признают экологических показателей. Эти и други. .  Этот и еще один эпизод экологической катастрофы - петиция с президентом Владимиром Путиным, а также .. bbc.com.   . BBC News . ( bbcrussian.com  ) . "Загрязнение воды - это очень далеко. Вот где-то. Проблема может сильно повлиять. ) В городе уже давно можно будет жить в нормальной форме. Это может стать одним из самых масштабных экологических проблем в регионе. Что известно о причинах этого инцидента и что может произойти в этом городе? - рассказывает корреспондент Русской службы Би-би-си. - На этой неделе жители Сегежа заявили о чрезвычайной ситуации, которую они называют экологическим катастрофой, и почему они жалуются на грязную воду, удушающий запах и не хватает воды, которая может быть связана с выбросом на

In [208]:
tokenized_new_article_karelia = word_tokenize(new_article_karelia, language="russian")

In [209]:
tokenized_articles_karelia = token_articles(articles_karelia) 

In [210]:
round(meteor_score.meteor_score(tokenized_articles_karelia, tokenized_new_article_karelia), 4)

0.2545

In [211]:
articles_google = get_articles(path='./tests/10/')

In [212]:
new_article_google = generate_text(articles_karelia)
new_article_google

'На прошлой неделе в Карелии выпал желтый снег, после чего стали массово гибнуть птицы. Местные власти заявили, что экологическая катастрофа в городе в норме. Власти считают, что все экологические показатели в Норме, однако не признают экологических показателей. Эти и други. .  Этот и еще один эпизод экологической катастрофы - петиция с президентом Владимиром Путиным, а также .. bbc.com.   . BBC News . ( bbcrussian.com  ) . "Загрязнение воды - это очень далеко. Вот где-то. Проблема может сильно повлиять. ) В городе уже давно можно будет жить в нормальной форме. Это может стать одним из самых масштабных экологических проблем в регионе. Что известно о причинах этого инцидента и что может произойти в этом городе? - рассказывает корреспондент Русской службы Би-би-си. - На этой неделе жители Сегежа заявили о чрезвычайной ситуации, которую они называют экологическим катастрофой, и почему они жалуются на грязную воду, удушающий запах и не хватает воды, которая может быть связана с выбросом на

In [213]:
tokenized_new_article_google = word_tokenize(new_article_google, language="russian")

In [214]:
tokenized_articles_google = token_articles(articles_google) 

In [215]:
round(meteor_score.meteor_score(tokenized_articles_google, tokenized_new_article_google), 4)

0.1794